In [56]:
# models_research_report_structured.py
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel, Field

# ---------- META ----------

class Analyst(BaseModel):
    name: str = Field(..., description="Tên chuyên viên phân tích")
    email: Optional[str] = Field(None, description="Email liên hệ (nếu có)")
    role: Optional[str] = Field(None, description="Vai trò: Lead, Junior,...")

class Meta(BaseModel):
    ticker: str = Field(..., description="Mã cổ phiếu, ví dụ: MSH")
    company_name: Optional[str] = Field(
        None, description="Tên công ty, ví dụ: CTCP May Sông Hồng"
    )
    exchange: Optional[str] = Field(
        None, description="Sàn niêm yết, ví dụ: HOSE, HNX, UPCOM"
    )
    sector: Optional[str] = Field(
        None, description="Ngành, ví dụ: Dệt may"
    )
    industry: Optional[str] = Field(
        None, description="Tiểu ngành / phân ngành chi tiết (nếu có)"
    )
    broker: Optional[str] = Field(
        None, description="Tên công ty chứng khoán phát hành báo cáo"
    )
    report_type: Optional[str] = Field(
        None,
        description="Loại báo cáo: Initiation, Company Update, Result Flash, Sector,...",
    )
    language: Optional[str] = Field("vi", description="Ngôn ngữ của báo cáo, vi/en")
    publish_date: Optional[str] = Field(
        None,
        description="Ngày phát hành báo cáo, định dạng YYYY-MM-DD nếu parse được",
    )
    analysts: List[Analyst] = Field(
        default_factory=list, description="Danh sách chuyên viên phụ trách báo cáo"
    )
    source_file: Optional[str] = Field(
        None, description="Tên file nguồn PDF hoặc path tương đối, nếu muốn lưu lại"
    )
    source_pages: Optional[str] = Field(
        None, description="Thông tin trang trích xuất (ví dụ: '1-12' hoặc '2,4-7')"
    )
    report_id: Optional[int] = Field(
        None, description="id để tải nội dung report"
    )

# ---------- RECOMMENDATION (BẢN GỌN, KHÔNG dict) ----------

class RecommendationItem(BaseModel):
    rating: str = Field(
        ...,
        description="Khuyến nghị chính: ví dụ BUY/HOLD/SELL hoặc MUA/GIỮ/BÁN."
    )
    target_price: Optional[float] = Field(
        None, description="Giá mục tiêu (VND/cp)"
    )
    upside_pct: Optional[float] = Field(
        None, description="Mức tăng/giảm kỳ vọng so với giá hiện tại, tính theo %"
    )
    time_horizon: Optional[str] = Field(
        None, description="Thời gian khuyến nghị, ví dụ: '12 tháng'"
    )
    catalysts: Optional[str] = Field(
        None, description="Các yếu tố tích cực/catalyst chính hỗ trợ luận điểm"
    )
    risks: Optional[str] = Field(
        None, description="Các rủi ro chính gắn với khuyến nghị này"
    )

class Recommendation(BaseModel):
    summary: str = Field(
        ...,
        description=(
            "Tóm tắt 1-3 câu về khuyến nghị đầu tư tổng thể: "
            "rating, target_price, upside, bối cảnh chính."
        ),
    )
    items: List[RecommendationItem] = Field(
        ...,
        description="Danh sách khuyến nghị chi tiết (thường 1 item).",
    )
    # rating_scale bỏ dict, dùng text cho nhẹ, tránh lỗi schema
    rating_scale_text: Optional[str] = Field(
        None,
        description="Nếu báo cáo có giải thích thang rating thì tóm tắt lại bằng 1 đoạn text.",
    )
    sector_view: Optional[str] = Field(
        None, description="Quan điểm chung về ngành (tích cực, trung lập, tiêu cực...)"
    )

# ---------- INVESTMENT THESIS ----------

class ThesisPillar(BaseModel):
    title: str = Field(..., description="Tiêu đề trụ cột luận điểm, ngắn gọn")
    description: str = Field(
        ...,
        description="Mô tả chi tiết trụ cột (1–3 câu): vì sao đây là driver chính.",
    )
    time_horizon: Optional[str] = Field(
        None, description="Khoảng thời gian luận điểm này có hiệu lực (nếu nêu)"
    )
    evidence: Optional[str] = Field(
        None,
        description="Các bằng chứng, số liệu, diễn giải được trích từ báo cáo",
    )
    key_metrics: Optional[List[str]] = Field(
        None,
        description="Các chỉ số định lượng chính liên quan (ví dụ: ['gross_margin', 'EPS'])",
    )

class InvestmentThesis(BaseModel):
    summary: Optional[str] = Field(
        None,
        description=(
            "Tổng quan luận điểm đầu tư (2–5 câu) – tại sao nên/không nên "
            "đầu tư cổ phiếu này theo quan điểm của CTCK."
        ),
    )
    pillars: List[ThesisPillar] = Field(
        default_factory=list,
        description="3–5 trụ cột chính của luận điểm đầu tư (nếu có thể trích ra).",
    )

# ---------- KEY POINTS ----------

class KeyPoints(BaseModel):
    positives: List[str] = Field(
        default_factory=list,
        description="Các ý tích cực nổi bật (bullet), mỗi phần tử 1 câu ngắn.",
    )
    negatives: List[str] = Field(
        default_factory=list,
        description="Các ý tiêu cực/điểm yếu chính (bullet).",
    )
    watch_list: List[str] = Field(
        default_factory=list,
        description="Các yếu tố cần theo dõi (watch-list): sự kiện, biến số vĩ mô, chính sách, v.v.",
    )

# ---------- ROOT MODEL DÙNG CHO RESPONSE_FORMAT ----------

class ResearchReportStructured(BaseModel):
    """
    Schema tối giản cho structured output:
    chỉ gồm meta, recommendation, investment_thesis, key_points.
    Các phần khác (risks, valuation, earnings_outlook, ...) để xử lý sau.
    """

    meta: Meta = Field(..., description="Thông tin nhận diện báo cáo")
    recommendation: Recommendation = Field(
        ..., description="Khuyến nghị đầu tư từ CTCK"
    )
    investment_thesis: Optional[InvestmentThesis] = Field(
        None, description="Luận điểm đầu tư chính"
    )
    key_points: Optional[KeyPoints] = Field(
        None, description="Các điểm nhấn chính: tích cực/tiêu cực/watch-list"
    )

    class Config:
        title = "ResearchReportStructured"
        str_strip_whitespace = True
        validate_assignment = True


C:\Users\hung.hm\AppData\Local\Temp\ipykernel_40276\1271232558.py:147: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class ResearchReportStructured(BaseModel):


In [57]:
# prompt_research_report.py
from langchain_core.prompts import ChatPromptTemplate

RESEARCH_REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Bạn là một chuyên gia phân tích chứng khoán Việt Nam.
Nhiệm vụ của bạn: ĐỌC KỸ nội dung báo cáo phân tích của công ty chứng khoán
và TRẢ VỀ một object JSON theo schema Pydantic `ResearchReport`.

RẤT QUAN TRỌNG:
- Chỉ trích xuất thông tin CÓ XUẤT HIỆN trong báo cáo.
- KHÔNG được bịa số liệu, ngày tháng, rating, target price, WACC,... nếu báo cáo không nêu.
- Nếu không chắc, hãy để field đó là null hoặc danh sách rỗng.
- Tập trung ƯU TIÊN cho 4 phần sau:
  1) meta
  2) recommendation
  3) investment_thesis
  4) key_points

Giải thích chi tiết:

1. meta
   - ticker: lấy mã cổ phiếu chính của báo cáo (vd: MSH).
   - company_name: tên đầy đủ công ty nếu có.
   - exchange: HOSE / HNX / UPCOM nếu có.
   - sector / industry: nếu báo cáo có nêu ngành/tập đoàn.
   - broker: tên công ty chứng khoán phát hành báo cáo.
   - report_type: Initiation / Company Update / Result Flash / Sector... nếu có.
   - publish_date: nếu có ngày phát hành trên báo cáo, hãy chuẩn hóa về dạng YYYY-MM-DD nếu có thể.
   - analysts: danh sách chuyên viên (tên, email, role nếu có).
   - source_file: giữ nguyên tên file được truyền vào.
   - source_pages: bạn có thể để null (sẽ xử lý sau bằng pipeline khác).
   - report_id: id để lấy nội dung report
2. recommendation
   - summary: tóm tắt 1–3 câu, phải gồm: rating, target_price, upside (%), thời gian (nếu có),
     và 1–2 ý về bối cảnh kết quả kinh doanh/góc nhìn chính.
   - items: thường chỉ 1 phần tử. Mỗi item:
       + rating: đúng như báo cáo (BUY/HOLD/SELL hoặc MUA/GIỮ/BÁN...). Không tự dịch sang tiếng khác.
       + target_price: số giá mục tiêu (VND/cp), nếu có.
       + upside_pct: phần trăm tăng/giảm so với giá hiện tại, nếu có.
       + time_horizon: ví dụ '12 tháng' nếu có.
       + catalysts: tóm tắt các yếu tố tích cực chính.
       + risks: tóm tắt các rủi ro chính liên quan tới khuyến nghị.
   - rating_scale, sector_view: CHỈ điền nếu báo cáo có nêu rõ; nếu không thì để null.

3. investment_thesis
   - summary: 2–5 câu tóm tắt luận điểm đầu tư tổng thể của báo cáo
     (vì sao nên/không nên đầu tư cổ phiếu này).
   - pillars: cố gắng rút ra 3–5 "trụ cột" rõ ràng, mỗi pillar:
       + title: 1 câu ngắn (ví dụ: "Biên LNG cải thiện nhờ đơn hàng CMT").
       + description: 1–3 câu giải thích cụ thể.
       + time_horizon: nếu tồn tại trong báo cáo (ngắn hạn/trung hạn/dài hạn hoặc năm cụ thể).
       + evidence: nếu có thể, trích dẫn ngắn gọn dữ liệu/sự kiện từ báo cáo.
       + key_metrics: danh sách tên chỉ số nếu trong phần này có đề cập (vd: ["gross_margin", "EPS"]).
   - Nếu báo cáo không có cấu trúc rõ ràng, hãy tự nhóm ý chính thành pillars logic
     nhưng KHÔNG được bịa thông tin mới.

4. key_points
   - positives: list các bullet "tích cực" (mỗi phần tử là 1 câu).
   - negatives: list các bullet "tiêu cực/hạn chế" (mỗi phần tử là 1 câu).
   - watch_list: list các yếu tố cần theo dõi (sự kiện, rủi ro, biến số vĩ mô, thuế quan, tỷ giá...).
   - Các bullet nên ngắn gọn, rõ ý, trích từ hoặc tổng hợp trực tiếp từ nội dung báo cáo.

Các phần khác trong schema (risks, valuation, earnings_outlook, business_profile, events_and_timeline):
- Bạn CHỈ điền nếu thông tin xuất hiện rõ ràng trong báo cáo.
- Nếu không chắc, hoặc không có dữ liệu, hãy để None / null / danh sách rỗng.

Đầu ra cuối cùng PHẢI là một JSON object khớp 100% với schema Pydantic ResearchReport.
Không thêm field lạ, không bớt field, không kèm giải thích bên ngoài JSON.

""",
        ),
        (
            "user",
            """
Bạn được cung cấp:
- Tên file báo cáo: {source_file}
- report_id: {report_id}
- Nội dung text đã được trích xuất từ PDF (có thể xuống dòng, có lỗi OCR nhẹ).

Nhiệm vụ: dựa trên nội dung này, hãy sinh ra object ResearchReport đúng schema.

--- BẮT ĐẦU NỘI DUNG BÁO CÁO ---
{report_text}
--- KẾT THÚC NỘI DUNG BÁO CÁO ---
""",
        ),
    ]
)


In [58]:
# parse_msh_report.py
from pathlib import Path
from typing import Optional
import requests
from pypdf import PdfReader
from langchain_openai import ChatOpenAI
from Class.auth import FireAnt
from sqlalchemy.orm import Session
from manage_engine import engine_stockDB
from Model.model_infoMarket import Report
from io import BytesIO
import re
def download_report_pdf_bytes(report_id: int, engine=None):
    """
    Tải PDF FireAnt → trả về (pdf_bytes, filename)
    filename có thể None nếu FireAnt không trả về
    """
    if engine is None:
        engine = engine_stockDB

    # --- Lấy real_id ---
    with Session(engine) as db:
        row = db.query(Report.report_id).filter(Report.id == report_id).first()
        if not row:
            raise ValueError(f"Không tìm thấy report id={report_id}")
        real_id = str(row[0])

    # --- Gọi FireAnt ---
    url = f"https://static.fireant.vn/reports/content/{real_id}"
    try:
        fa = FireAnt()
        headers = fa.headers()
        r = requests.get(url, headers=headers, timeout=20)

        if r.status_code != 200:
            raise RuntimeError(f"Tải file thất bại, code={r.status_code}")

        # --- Lấy filename từ Content-Disposition ---
        filename = None
        cd = r.headers.get("content-disposition")
        if cd:
            match = re.search(r'filename="?([^"]+)"?', cd)
            if match:
                filename = match.group(1)

        # fallback nếu không có header filename:
        if filename is None:
            filename = f"{report_id}.pdf"

        return r.content, filename

    except Exception as e:
        raise RuntimeError(f"Lỗi tải FireAnt PDF: {e}") from e




def extract_pdf_text_from_bytes(pdf_bytes: bytes, max_pages: Optional[int] = None) -> str:
    """
    Đọc PDF từ bytes và nối text theo trang.
    """
    reader = PdfReader(BytesIO(pdf_bytes))
    texts = []

    for i, page in enumerate(reader.pages):
        if max_pages is not None and i >= max_pages:
            break
        txt = page.extract_text() or ""
        texts.append(f"\n===== PAGE {i+1} =====\n{txt}")

    return "\n".join(texts)


def parse_research_report_from_pdf(
    report_id=int,
    model_name: str = "gpt-4.1-mini",  # hoặc "gpt-4o-mini" tuỳ bạn
    temperature: float = 0.0,
    max_pages: Optional[int] = 15,
) -> ResearchReportStructured:
    # 1. Extract text
    pdf_bytes,filename=download_report_pdf_bytes(report_id)
    report_text = extract_pdf_text_from_bytes(pdf_bytes, max_pages=max_pages)
    source_file = filename if filename is None else f'REPORT_ID_{report_id}'

    # 2. Khởi tạo LLM + structured output
    llm = ChatOpenAI(model=model_name, temperature=temperature)
    structured_llm = llm.with_structured_output(ResearchReportStructured)

    # 3. Tạo chain: prompt -> structured_llm
    chain = RESEARCH_REPORT_PROMPT | structured_llm

    # 4. Gọi chain
    result: ResearchReportStructured = chain.invoke(
        {
            "source_file": source_file,
            "report_id":report_id,
            "report_text": report_text,
        }
    )
    return result


# if __name__ == "__main__":
#     pdf_path = "PVT.pdf"  # đổi lại path trong hệ thống của bạn

#     report = parse_research_report_from_pdf(pdf_path)
#     # debug nhẹ:
#     print("Ticker:", report.meta.ticker)
#     print("Broker:", report.meta.broker)
#     print("Rating:", report.recommendation.items[0].rating)
#     print("Target price:", report.recommendation.items[0].target_price)
#     print("Thesis summary:", report.investment_thesis.summary if report.investment_thesis else None)
#     print("Positives:", report.key_points.positives if report.key_points else None)


In [59]:
result=parse_research_report_from_pdf(394670)
result

ResearchReportStructured(meta=Meta(ticker='QNS', company_name='CTCP Đường Quảng Ngãi', exchange=None, sector=None, industry=None, broker='Chứng khoán Mirae Asset', report_type='Company Update', language='vi', publish_date=None, analysts=[Analyst(name='Huỳnh Thị Thu Thảo', email='thao.htt@miraeasset.com.vn', role=None)], source_file='REPORT_ID_394670', source_pages=None, report_id=394670), recommendation=Recommendation(summary='Khuyến nghị Mua với giá mục tiêu 57.900 VND/cp, tiềm năng tăng giá 31.6% trong 12 tháng. Kết quả kinh doanh tích cực nhờ mảng sữa đậu nành tăng trưởng bền vững và cổ tức tiền mặt hấp dẫn gần 9%.', items=[RecommendationItem(rating='Mua', target_price=57900.0, upside_pct=31.6, time_horizon='12 tháng', catalysts='Mảng sữa đậu nành tăng trưởng 7%/năm, giá đậu nành thấp hỗ trợ biên lợi nhuận, dự án mở rộng công suất và nhà máy Ethanol giúp tăng trưởng bền vững.', risks='Rủi ro dài hạn từ biến động giá đậu nành nếu chi phí tăng không chuyển được cho người tiêu dùng.')]

In [71]:
import json
report_id=394670
ticker_db="QNS"
publish_date_db='2022-01-01'



report_id=report_id 
ticker=result.meta.ticker if result.meta.ticker is not None else ticker_db
publish_date=result.meta.publish_date if result.meta.publish_date  else publish_date_db
if len(result.recommendation.items)>0:
    rating=result.recommendation.items[0].rating.upper()
    target_price=result.recommendation.items[0].target_price
else:
    rating=None 
    target_price=None
content_json=result.model_dump()


In [73]:
content_json

{'meta': {'ticker': 'QNS',
  'company_name': 'CTCP Đường Quảng Ngãi',
  'exchange': None,
  'sector': None,
  'industry': None,
  'broker': 'Chứng khoán Mirae Asset',
  'report_type': 'Company Update',
  'language': 'vi',
  'publish_date': None,
  'analysts': [{'name': 'Huỳnh Thị Thu Thảo',
    'email': 'thao.htt@miraeasset.com.vn',
    'role': None}],
  'source_file': 'REPORT_ID_394670',
  'source_pages': None,
  'report_id': 394670},
 'recommendation': {'summary': 'Khuyến nghị Mua với giá mục tiêu 57.900 VND/cp, tiềm năng tăng giá 31.6% trong 12 tháng. Kết quả kinh doanh tích cực nhờ mảng sữa đậu nành tăng trưởng bền vững và cổ tức tiền mặt hấp dẫn gần 9%.',
  'items': [{'rating': 'Mua',
    'target_price': 57900.0,
    'upside_pct': 31.6,
    'time_horizon': '12 tháng',
    'catalysts': 'Mảng sữa đậu nành tăng trưởng 7%/năm, giá đậu nành thấp hỗ trợ biên lợi nhuận, dự án mở rộng công suất và nhà máy Ethanol giúp tăng trưởng bền vững.',
    'risks': 'Rủi ro dài hạn từ biến động giá đ

In [64]:
import json

# result is a Pydantic model instance; convert to dict then pretty-print JSON
print(json.dumps(result.model_dump(), ensure_ascii=False, indent=2))

{
  "meta": {
    "ticker": "QNS",
    "company_name": "CTCP Đường Quảng Ngãi",
    "exchange": null,
    "sector": null,
    "industry": null,
    "broker": "Chứng khoán Mirae Asset",
    "report_type": "Company Update",
    "language": "vi",
    "publish_date": null,
    "analysts": [
      {
        "name": "Huỳnh Thị Thu Thảo",
        "email": "thao.htt@miraeasset.com.vn",
        "role": null
      }
    ],
    "source_file": "REPORT_ID_394670",
    "source_pages": null,
    "report_id": 394670
  },
  "recommendation": {
    "summary": "Khuyến nghị Mua với giá mục tiêu 57.900 VND/cp, tiềm năng tăng giá 31.6% trong 12 tháng. Kết quả kinh doanh tích cực nhờ mảng sữa đậu nành tăng trưởng bền vững và cổ tức tiền mặt hấp dẫn gần 9%.",
    "items": [
      {
        "rating": "Mua",
        "target_price": 57900.0,
        "upside_pct": 31.6,
        "time_horizon": "12 tháng",
        "catalysts": "Mảng sữa đậu nành tăng trưởng 7%/năm, giá đậu nành thấp hỗ trợ biên lợi nhuận, dự án mở

In [39]:
RENDER_MARKDOWN_PROMPT = """
Bạn là chuyên gia phân tích chứng khoán. 
Nhiệm vụ: Dựa vào dữ liệu JSON bên dưới (đúng theo schema ResearchReportStructured),
hãy tạo báo cáo phân tích dạng MARKDOWN theo TEMPLATE chuẩn.

YÊU CẦU:
- KHÔNG thêm thông tin không có trong JSON.
- KHÔNG phóng đại, không suy diễn.
- Nếu thiếu dữ liệu, để trống hoặc bỏ dòng đó.
- Văn phong: Học thuật – chuyên nghiệp – cô đọng.
- Format phải giống mẫu PVT.

--- JSON REPORT ---
{json_report}
--- END JSON REPORT ---

--- TEMPLATE MARKDOWN ---
🔎 **BÁO CÁO PHÂN TÍCH – {{TICKER}} ({{COMPANY_NAME}})**  
**Khuyến nghị:** {{RATING}} • **Giá mục tiêu:** {{TARGET_PRICE}} VNĐ/cp • **Upside:** {{UPSIDE}}% - {{publish_date}}

---

# 1. Kết luận đầu tư
{{THESIS_SUMMARY}}

---

# 2. Khuyến nghị & Giá mục tiêu

## 2.1. Khuyến nghị {{RATING}}
- **Giá mục tiêu:** {{TARGET_PRICE}} VNĐ/cp  
- **Upside 12 tháng:** {{UPSIDE}}%  
- **Đánh giá:** {{RECOMMENDATION_SUMMARY}}

## 2.2. Yếu tố hỗ trợ
{{POSITIVE_BULLETS}}

## 2.3. Rủi ro
{{NEGATIVE_BULLETS}}

---

# 3. Các trụ cột đầu tư

Hãy diễn đạt lại mỗi trụ cột đầu tư thành **một đoạn văn trôi chảy**, gồm:
- Ý chính của pillar (title)
- Diễn giải (description)
- Gài tự nhiên thông tin dạng bằng chứng/evidence (nếu có)
- Không nhắc tới từ “bằng chứng”, “chỉ số chính”, “thời hạn”
- Không liệt kê key_metrics
- Viết như một đoạn bình luận phân tích của chuyên viên

Định dạng mỗi trụ cột:
## 3.{{pillar_index}}. {{title}}
{{paragraph}}

Sau đây là danh sách trụ cột cần viết lại theo dạng văn mạch lạc:
{{PILLARS_SECTION}}

---

# 4. Các điểm nhấn tích cực & tiêu cực

## 4.1. 🟢 Điểm tích cực
{{POSITIVES}}

## 4.2. 🔴 Điểm tiêu cực
{{NEGATIVES}}

## 4.3. Danh mục theo dõi
{{WATCH_LIST}}

---

# 5. Kết luận
{{FINAL_CONCLUSION}}
--- END TEMPLATE ---

Hãy thay thế {{...}} bằng nội dung phù hợp từ JSON. 
Trả về DUY NHẤT markdown.
"""


In [40]:
import json
from langchain_openai import ChatOpenAI

def render_markdown_from_report(report_structured):
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)

    json_report = json.dumps(report_structured, ensure_ascii=False, indent=2) \
        if isinstance(report_structured, dict) \
        else json.dumps(report_structured.model_dump(), ensure_ascii=False, indent=2)

    prompt = RENDER_MARKDOWN_PROMPT.format(json_report=json_report)
    resp = llm.invoke(prompt)
    return resp.content


In [41]:
content=render_markdown_from_report(report)
content

'🔎 **BÁO CÁO PHÂN TÍCH – PVT (Tổng CTCP Vận tải Dầu khí)**  \n**Khuyến nghị:** KHẢ QUAN • **Giá mục tiêu:** 22,000 VNĐ/cp • **Upside:** 22.2% - 2025-11-10\n\n---\n\n# 1. Kết luận đầu tư\nPVT là doanh nghiệp hàng đầu trong lĩnh vực vận tải dầu khí tại Việt Nam với nền tảng tài chính lành mạnh và tăng trưởng doanh thu ổn định. Mặc dù lợi nhuận ròng 9T2025 giảm so với cùng kỳ do biên lợi nhuận thu hẹp, công ty vẫn duy trì chiến lược mở rộng đội tàu hợp lý nhằm tận dụng nhu cầu vận tải nội địa và khu vực. Định giá hiện tại hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm, tạo cơ hội tích lũy cổ phiếu.\n\n---\n\n# 2. Khuyến nghị & Giá mục tiêu\n\n## 2.1. Khuyến nghị KHẢ QUAN\n- **Giá mục tiêu:** 22,000 VNĐ/cp  \n- **Upside 12 tháng:** 22.2%  \n- **Đánh giá:** Khuyến nghị KHẢ QUAN với giá mục tiêu 22,000 VNĐ/cp, tiềm năng tăng giá 22.2%. Lợi nhuận ròng 2025-2027 dự kiến tăng trưởng nhẹ với động lực chính từ mở rộng đội tàu bù đắp cho khả năng giảm nhẹ giá cước vận tải.\n\n## 2.2. Yếu tố h

In [42]:
print(content)

🔎 **BÁO CÁO PHÂN TÍCH – PVT (Tổng CTCP Vận tải Dầu khí)**  
**Khuyến nghị:** KHẢ QUAN • **Giá mục tiêu:** 22,000 VNĐ/cp • **Upside:** 22.2% - 2025-11-10

---

# 1. Kết luận đầu tư
PVT là doanh nghiệp hàng đầu trong lĩnh vực vận tải dầu khí tại Việt Nam với nền tảng tài chính lành mạnh và tăng trưởng doanh thu ổn định. Mặc dù lợi nhuận ròng 9T2025 giảm so với cùng kỳ do biên lợi nhuận thu hẹp, công ty vẫn duy trì chiến lược mở rộng đội tàu hợp lý nhằm tận dụng nhu cầu vận tải nội địa và khu vực. Định giá hiện tại hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm, tạo cơ hội tích lũy cổ phiếu.

---

# 2. Khuyến nghị & Giá mục tiêu

## 2.1. Khuyến nghị KHẢ QUAN
- **Giá mục tiêu:** 22,000 VNĐ/cp  
- **Upside 12 tháng:** 22.2%  
- **Đánh giá:** Khuyến nghị KHẢ QUAN với giá mục tiêu 22,000 VNĐ/cp, tiềm năng tăng giá 22.2%. Lợi nhuận ròng 2025-2027 dự kiến tăng trưởng nhẹ với động lực chính từ mở rộng đội tàu bù đắp cho khả năng giảm nhẹ giá cước vận tải.

## 2.2. Yếu tố hỗ trợ
- PVT là doa

🔎 **BÁO CÁO PHÂN TÍCH – PVT (Tổng CTCP Vận tải Dầu khí)**  
**Khuyến nghị:** KHẢ QUAN • **Giá mục tiêu:** 22,000 VNĐ/cp • **Upside:** 22.2%

---

# 1. Kết luận đầu tư  
PVT là doanh nghiệp hàng đầu trong lĩnh vực vận tải dầu khí tại Việt Nam với nền tảng tài chính lành mạnh và tăng trưởng doanh thu ổn định. Mặc dù lợi nhuận ròng 9T2025 giảm so với cùng kỳ do biên lợi nhuận thu hẹp, công ty vẫn duy trì chiến lược mở rộng đội tàu hợp lý nhằm tận dụng nhu cầu vận tải nội địa và khu vực. Định giá hiện tại hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm, tạo cơ hội tích lũy cổ phiếu.

---

# 2. Khuyến nghị & Giá mục tiêu

## 2.1. Khuyến nghị KHẢ QUAN
- **Giá mục tiêu:** 22,000 VNĐ/cp  
- **Upside 12 tháng:** 22.2%  
- **Đánh giá:** Khuyến nghị KHẢ QUAN với giá mục tiêu 22,000 VNĐ/cp, tiềm năng tăng giá 22.2%. Lợi nhuận ròng 2025-2027 dự kiến tăng trưởng nhẹ với động lực chính từ mở rộng đội tàu bù đắp cho khả năng giảm nhẹ giá cước vận tải.

## 2.2. Yếu tố hỗ trợ
- PVT là doanh nghiệp hàng đầu về vận tải dầu khí tại Việt Nam với nền tảng tài chính lành mạnh.  
- Doanh thu 9T2025 tăng 36.2% so với cùng kỳ nhờ mở rộng mảng vận tải và thương mại.  
- Chiến lược mở rộng đội tàu hợp lý, ưu tiên tàu chở dầu thành phẩm và hóa chất có biên lợi nhuận cao.  
- Định giá cổ phiếu hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm.  
- Dự báo lợi nhuận ròng 2025-2027 tăng trưởng nhẹ nhờ cải thiện biên lợi nhuận từ 2026.

## 2.3. Rủi ro
- Lợi nhuận ròng Q3/2025 giảm 33.1% so với cùng kỳ do biên lợi nhuận thu hẹp và chi phí tài chính tăng.  
- Tiến độ mở rộng đội tàu chậm hơn kế hoạch do giá tàu neo cao và chi phí đầu tư tăng.  
- Biên lợi nhuận gộp giảm do chi phí dịch vụ mua ngoài tăng cao khi mở rộng khai thác tuyến quốc tế.  
- Giá cước vận tải dầu thô chịu áp lực giảm do nhu cầu suy yếu và cạnh tranh từ tàu cỡ lớn.  
- Rủi ro từ chi phí đầu tư vượt dự kiến và xung đột thương mại toàn cầu ảnh hưởng nhu cầu vận tải.

---

# 3. Các trụ cột đầu tư

## 3.1. Mở rộng đội tàu làm động lực tăng trưởng doanh thu  
PVT đặt mục tiêu nâng quy mô đội tàu lên khoảng 100 chiếc vào năm 2030, ưu tiên đầu tư vào tàu chở dầu thành phẩm và hóa chất có biên lợi nhuận cao và ít cạnh tranh hơn. Tính đến cuối 2024, công ty sở hữu 58 tàu và dự kiến đạt khoảng 80 tàu vào cuối năm 2030 do chi phí đầu tư cao và tiến độ mở rộng chậm hơn kỳ vọng.

## 3.2. Tăng trưởng doanh thu vận tải dầu thành phẩm và hóa chất  
Mảng vận tải dầu thành phẩm và hóa chất được kỳ vọng tăng trưởng với tốc độ CAGR khoảng 8.4% nhờ mở rộng thương mại hóa chất tại châu Á, mang lại biên lợi nhuận tốt hơn so với vận tải dầu thô và hàng rời. Doanh thu vận tải dầu thành phẩm và hóa chất tăng tích cực trong bối cảnh các tuyến hóa chất và dầu thành phẩm châu Á có mức độ chuyên biệt và ít cạnh tranh.

## 3.3. Định giá hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm  
PVT hiện giao dịch với P/E 7.5 và P/B 1.0, thấp hơn mức trung bình 5 năm lần lượt là 9.0 và 1.2, cho thấy cổ phiếu đang ở vùng giá hấp dẫn để tích lũy. Giá cổ phiếu đã giảm 25.6% so với đỉnh 2024, EV/EBITDA cũng thấp hơn 16.3% so với trung bình 5 năm.

## 3.4. Áp lực biên lợi nhuận thu hẹp do chi phí tăng và giá cước giảm  
Biên lợi nhuận gộp giảm do chi phí dịch vụ mua ngoài tăng cao, đặc biệt khi công ty mở rộng khai thác tuyến quốc tế với chi phí thuê tàu ngoài và bảo dưỡng tăng. Biên lợi nhuận gộp quý 3/2025 giảm 8.3 điểm phần trăm so với cùng kỳ, trong khi chi phí tài chính tăng 37.4% do lỗ chênh lệch tỷ giá.

---

# 4. Các điểm nhấn tích cực & tiêu cực

## 4.1. 🟢 Điểm tích cực
- PVT là doanh nghiệp hàng đầu về vận tải dầu khí tại Việt Nam với nền tảng tài chính lành mạnh.  
- Doanh thu 9T2025 tăng 36.2% so với cùng kỳ nhờ mở rộng mảng vận tải và thương mại.  
- Chiến lược mở rộng đội tàu hợp lý, ưu tiên tàu chở dầu thành phẩm và hóa chất có biên lợi nhuận cao.  
- Định giá cổ phiếu hấp dẫn với P/E và P/B thấp hơn trung bình 5 năm.  
- Dự báo lợi nhuận ròng 2025-2027 tăng trưởng nhẹ nhờ cải thiện biên lợi nhuận từ 2026.

## 4.2. 🔴 Điểm tiêu cực
- Lợi nhuận ròng Q3/2025 giảm 33.1% so với cùng kỳ do biên lợi nhuận thu hẹp và chi phí tài chính tăng.  
- Tiến độ mở rộng đội tàu chậm hơn kế hoạch do giá tàu neo cao và chi phí đầu tư tăng.  
- Biên lợi nhuận gộp giảm do chi phí dịch vụ mua ngoài tăng cao khi mở rộng khai thác tuyến quốc tế.  
- Giá cước vận tải dầu thô chịu áp lực giảm do nhu cầu suy yếu và cạnh tranh từ tàu cỡ lớn.  
- Rủi ro từ chi phí đầu tư vượt dự kiến và xung đột thương mại toàn cầu ảnh hưởng nhu cầu vận tải.

## 4.3. Danh mục theo dõi
- Tiến độ mở rộng đội tàu và chi phí đầu tư trong các quý tới.  
- Diễn biến giá cước vận tải dầu thô và dầu thành phẩm trên thị trường quốc tế.  
- Tác động của xung đột thương mại và thuế quan toàn cầu đến nhu cầu vận tải.  
- Biến động chi phí dịch vụ mua ngoài, bảo dưỡng và nhiên liệu.  
- Kết quả kinh doanh quý 4/2025 và năm 2026.

---

# 5. Kết luận  
PVT duy trì vị thế dẫn đầu trong ngành vận tải dầu khí với chiến lược mở rộng đội tàu tập trung vào các phân khúc có biên lợi nhuận cao. Mặc dù chịu áp lực biên lợi nhuận trong ngắn hạn do chi phí tăng và giá cước giảm, công ty vẫn có nền tảng tài chính vững chắc và định giá hấp dẫn. Tiềm năng tăng trưởng doanh thu và lợi nhuận trong trung và dài hạn được hỗ trợ bởi mở rộng đội tàu và tăng trưởng mảng vận tải dầu thành phẩm, hóa chất. Nhà đầu tư nên theo dõi sát tiến độ mở rộng đội tàu, diễn biến giá cước và các rủi ro liên quan chi phí đầu tư cũng như môi trường thương mại toàn cầu.

---